In [2]:
import numpy as np
import cosmoprimo
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [59]:
def hmc_sample(U, grad_U, epsilon, L, current_q, std_dev):
    '''
         U: function returns the potential energy given a state q
         grad_u: function returns gradient of U given q
         epsilon: step size
         L: number of Leapfrog steps
         current_q: current generalized state trajectory starts from
         std_dev: vector of standard deviations for Gaussian (hyperparameter)
    '''
    q = current_q
    p = np.random.normal(np.zeros(len(q)), std_dev)  # sample zero-mean Gaussian
    current_p = p

    # Leapfrog: half step for momentum
    p = p - epsilon * grad_U(q) / 2

    for i in range(0, L):
        # Leapfrog: full step for position
        q = q + epsilon * p

        # Leapfrog: combine 2 half-steps for momentum across iterations
        if (i != L-1):
            p = p - epsilon * grad_U(q)

    # Leapfrog: final half step for momentum
    p = p - epsilon * grad_U(q)

    # Negate trajectory to make proposal symmetric (a no-op)
    p = -p

    # Compute potential and kinetic energies
    current_U = U(current_q)
    current_K = np.sum(current_p ** 2) / 2
    proposed_U = U(q)
    proposed_K = np.sum(p ** 2) / 2
    
    # Accept with probability specified using Equation 44:
    if np.random.rand() < np.exp(current_U - proposed_U + current_K - proposed_K):
        return q, proposed_U,1
    else:
        return current_q, current_U,0
def mh_sampler(f, proposal, steps, initial=0.0):
    x_curr = initial
    while True:
        accept = 0
        for i in range(steps):
            x_next = proposal(x_curr)
            if min(1, np.exp(-f(x_next) + f(x_curr))) > np.random.uniform(0, 1):
                x_curr = x_next
                accept += 1
            loglike = f(x_curr)
        yield x_curr,loglike,accept / steps

In [60]:
dr1_data = pd.read_csv('Practices/BAOdata/dr1_mean.txt', comment='#', delim_whitespace=True, header=None)
dr1_cov = np.loadtxt('Practices/BAOdata/dr1_cov.txt')
dr2_data = pd.read_csv('Practices/BAOdata/dr2_mean.txt', comment='#', delim_whitespace=True, header=None)
dr2_cov = np.loadtxt('Practices/BAOdata/dr2_cov.txt')

In [62]:
from cosmoprimo import Cosmology
from scipy.stats import norm

cosmo = Cosmology(engine='class') # LCDM
#cosmo = cosmo.clone(w0_fld=-0.42, wa_fld=-1.75) # w0waCDM

def chisq(q):
    theta = q.copy()
    hr_d = theta.pop('hr_d')
    cosmo_vary = cosmo.clone(**theta)
    speed_of_light = 2.99792458e5  # in km/s
    ba = cosmo_vary.get_background()
    
    # Model prediction
    def get_model(values, data):
        ind = data[2].values == values
        redshifts = data[0].values[ind]
        if values == 'DV_over_rs':
            model = (ba.comoving_angular_distance(redshifts) ** 2 / cosmo_vary.get('h') ** 2\
                      *  speed_of_light * redshifts / ba.hubble_function(redshifts))**(1/3) *cosmo_vary.get('h')/ hr_d
        elif values == 'DM_over_rs':
            model = ba.comoving_angular_distance(redshifts) / hr_d
        elif values == 'DH_over_rs':
            model = speed_of_light * cosmo_vary.get('h') / (ba.hubble_function(redshifts) * hr_d)
        return model, ind
    def set_model_vector(data):
        model_vec = np.zeros(len(data))
        dv_model, dv_zind = get_model('DV_over_rs', data)
        dm_model, dm_zind = get_model('DM_over_rs', data)
        dh_model, dh_zind = get_model('DH_over_rs', data)
        model_vec[dv_zind] = dv_model
        model_vec[dm_zind] = dm_model
        model_vec[dh_zind] = dh_model
        return model_vec
    model_dr1 = set_model_vector(dr1_data)
    model_dr2 = set_model_vector(dr2_data)
    # Chi-squared calculation
    delta_dr1 = dr1_data[1].values - model_dr1
    delta_dr2 = dr2_data[1].values - model_dr2
    chi2_dr1 = delta_dr1.T @ np.linalg.inv(dr1_cov) @ delta_dr1
    chi2_dr2 = delta_dr2.T @ np.linalg.inv(dr2_cov) @ delta_dr2
    return chi2_dr2 + chi2_dr1

def prior(theta):
    for key, value in theta.items():
        if key == 'Omega_cdm' and not (0.1 < value < 0.5):
            return 0
        if key == 'Omega_b' and not (0.01 < value < 0.1):
            return 0
        if key == 'h' and not (0.5 < value < 0.9):
            return 0
        return 1
        
def U(theta): 
    if prior(theta) == 0:
        return -np.inf
    chi2 = chisq(theta)
    #print(0.5 * (chi2 + prior_val))
    return 0.5 * (chi2)

def gradU(theta, h=1e-5):
    grad = {}
    base_U = U(theta)
    for key in theta.keys():
        theta_h = theta.copy()
        theta_h[key] = theta[key] + h
        
        #print(theta_h[key] - theta[key])
        U_h = U(theta_h)     
        grad[key] = (U_h - base_U) / h
    #print(grad)
    return pd.Series(grad)


In [92]:
sample = []
accepts = 0
epsilon = 3e-3
L = 30; N = 300
current_q = pd.Series({'Omega_m' : 0.297, 'hr_d' : 101.4}) + pd.Series({'Omega_m' : np.random.normal(0,0.01), 'hr_d' : np.random.normal(0,0.1)})
std_dev = [1.,1.]

samples= []; likelihoods=[]
accepts = 0

for i in tqdm(range(0, N,1)):
    sample, like,accept = hmc_sample(U, gradU, epsilon, L, current_q, std_dev)
    samples.append(sample); likelihoods.append(like)
    accepts += accept
    current_q = samples[-1]
    

  0%|                                                                                                                                  | 0/300 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [07:54<00:00,  1.58s/it]


In [93]:
import matplotlib.pyplot as plt
samples = pd.DataFrame(samples)
plt.figure(figsize=(8,6))
plt.scatter(samples['Omega_m'], samples['hr_d'])
plt.savefig('img.png')      
print(accepts/N)

0.82


In [ ]:
from getdist import MCSamples, plots
samples1 = pd.read_csv('Practices/mcmc/HMC_DESI_samples_1.csv')
loglikes1 = np.load('Practices/mcmc/HMC_DESI_likelihoods_1.npy')
accepts1 = np.loadtxt('Practices/mcmc/HMC_DESI_acceptance_num_1.txt')
print('acceptance ratio:', accepts1/1000)
MCsamples_arr = np.array([samples1['Omega_m'].values, samples1['hr_d'].values]).T
chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes1, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
stats = chains.getMargeStats()
print(stats)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, ['Omega_m', 'hr_d'], filled=True,lims=[0.265, 0.33,98.5,104.5])#, lims=[0.28,0.345,98.7, 102.5]
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_HMC_2d_1.pdf')
print('L : 30, epsilon : 3e-3, std_dev : [1.,1.]')
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))

acceptance ratio: 0.708
Removed no burn in
Marginalized limits: 0.68; 0.95; 0.99

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     2.9630629E-01  1.0609147E-02  2.8173588E-01  3.0965406E-01  two    2.7759720E-01  3.1356662E-01  two    2.7531879E-01  3.1625137E-01  two     \Omega_m
hr_d        1.0163234E+02  7.6582859E-01  1.0065850E+02  1.0205557E+02  two    1.0040513E+02  1.0322426E+02  two    1.0012533E+02  1.0338699E+02  two     h r_d

L : 30, epsilon : 3e-3, std_dev : [1.,1.]
\Omega_m = 0.296\pm 0.011 h r_d = 101.63^{+0.42}_{-0.97}
\Omega_m = 0.296^{+0.017}_{-0.019} h r_d = 101.6^{+1.6}_{-1.2}
\Omega_m = 0.296^{+0.020}_{-0.021} h r_d = 101.6^{+1.8}_{-1.5}


In [105]:
samples2 = pd.read_csv('Practices/mcmc/HMC_DESI_samples_2.csv')
loglikes2 = np.load('Practices/mcmc/HMC_DESI_likelihoods_2.npy')
accepts2 = np.loadtxt('Practices/mcmc/HMC_DESI_acceptance_num_2.txt')
print('acceptance ratio:', accepts2/1000)
MCsamples_arr = np.array([samples2['Omega_m'].values, samples2['hr_d'].values]).T
chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes2, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
stats=chains.getMargeStats()
print(stats)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, ['Omega_m', 'hr_d'], filled=True, lims=[0.265, 0.33,98.5,104.5])
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_HMC_2d_2.pdf')
print('L : 15, epsilon : 3e-3, std_dev : [1.,1.]')
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))

acceptance ratio: 0.947
Removed no burn in
Marginalized limits: 0.68; 0.95; 0.99

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     3.0008034E-01  3.5687350E-03  2.9662307E-01  3.0368774E-01  two    2.9302718E-01  3.0705554E-01  two    2.9073629E-01  3.0859705E-01  two     \Omega_m
hr_d        1.0128644E+02  1.8419691E-01  1.0108905E+02  1.0149036E+02  two    1.0090351E+02  1.0166512E+02  two    1.0079628E+02  1.0176709E+02  two     h r_d

L : 15, epsilon : 3e-3, std_dev : [1.,1.]
\Omega_m = 0.3001\pm 0.0036 h r_d = 101.29\pm 0.18
\Omega_m = 0.3001^{+0.0070}_{-0.0071} h r_d = 101.29^{+0.38}_{-0.38}
\Omega_m = 0.3001^{+0.0085}_{-0.0093} h r_d = 101.29^{+0.48}_{-0.49}


In [104]:
from getdist import MCSamples, plots
samples3 = pd.read_csv('Practices/mcmc/HMC_DESI_samples_3.csv')
loglikes3 = np.load('Practices/mcmc/HMC_DESI_likelihoods_3.npy')
accepts3 = np.loadtxt('Practices/mcmc/HMC_DESI_acceptance_num_3.txt')
print('acceptance ratio:', accepts3/1000)
MCsamples_arr = np.array([samples3['Omega_m'].values, samples3['hr_d'].values]).T
chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes3, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
stats=chains.getLikeStats()
print(stats)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, ['Omega_m', 'hr_d'], filled=True, lims=[0.265, 0.33,98.5,104.5])
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_HMC_2d_3.pdf')
print('L : 15, epsilon : 1e-3, std_dev : [1.,1.]')
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))

acceptance ratio: 0.283
Removed no burn in
Best fit sample -log(Like) = 5.148842
mean(-Ln(like)) = 34.987107
-Ln(mean like)  = 6.900927
2*Var(Ln(like)) = 1401.520625

parameter   bestfit        lower1         upper1         lower2         upper2
Omega_m     2.9812162E-01  2.6866421E-01  3.2873943E-01  2.6866421E-01  3.2873943E-01   \Omega_m
hr_d        1.0148585E+02  9.9914849E+01  1.0252786E+02  9.9914849E+01  1.0252786E+02   h r_d



L : 15, epsilon : 1e-3, std_dev : [1.,1.]
\Omega_m = 0.311^{+0.018}_{-0.042} h r_d = 101.58^{+0.59}_{-1.3}
\Omega_m = 0.311^{+0.018}_{-0.042} h r_d = 101.58^{+0.61}_{-1.5}
\Omega_m = 0.311^{+0.018}_{-0.042} h r_d = 101.58^{+0.78}_{-1.6}


In [113]:
samples4 = pd.read_csv('Practices/mcmc/HMC_DESI_samples_4.csv')
loglikes4 = np.load('Practices/mcmc/HMC_DESI_likelihoods_4.npy')
accepts4 = np.loadtxt('Practices/mcmc/HMC_DESI_acceptance_num_4.txt')
print('acceptance ratio:', accepts4/1000)
MCsamples_arr = np.array([samples4['Omega_m'].values, samples4['hr_d'].values]).T
chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes4, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
chains.updateSettings({'contours': [0.68,0.95,0.997]})
stats = chains.getMargeStats()
print(stats)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, ['Omega_m', 'hr_d'], filled=True, lims=[0.265, 0.33,98.5,104.5])
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_HMC_2d_4.pdf')
print('L : 30, epsilon : 1e-3, std_dev : [2.,2.]')
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))
plt.close('all')

acceptance ratio: 0.877
Removed no burn in
Marginalized limits: 0.68; 0.95; 0.997

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     2.9706245E-01  9.5428931E-03  2.8531401E-01  3.0990899E-01  two    2.8161226E-01  3.1532683E-01  two    2.7608708E-01  3.2317160E-01  two     \Omega_m
hr_d        1.0159145E+02  8.1523610E-01  1.0041132E+02  1.0264293E+02  two    1.0005720E+02  1.0285148E+02  two    9.9840881E+01  1.0300489E+02  two     h r_d

L : 30, epsilon : 1e-3, std_dev : [2.,2.]
\Omega_m = 0.2971\pm 0.0095 h r_d = 101.6^{+1.1}_{-1.2}
\Omega_m = 0.297^{+0.018}_{-0.015} h r_d = 101.6^{+1.3}_{-1.5}
\Omega_m = 0.297^{+0.026}_{-0.021} h r_d = 101.6^{+1.4}_{-1.8}


In [110]:
samples5 = pd.read_csv('Practices/mcmc/HMC_DESI_samples_5.csv')
loglikes5 = np.load('Practices/mcmc/HMC_DESI_likelihoods_5.npy')
accepts5 = np.loadtxt('Practices/mcmc/HMC_DESI_acceptance_num_5.txt')
print('acceptance ratio:', accepts5/7000)
MCsamples_arr = np.array([samples5['Omega_m'].values, samples5['hr_d'].values]).T
chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes5, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
chains.updateSettings({'contours': [0.68,0.95,0.997]})
stats = chains.getMargeStats()
print(stats)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, ['Omega_m', 'hr_d'], filled=True, lims=[0.265, 0.33,98.5,104.5])
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_HMC_2d_5.pdf')
print('L : 30, epsilon : 3e-3, std_dev : [1.,1.]')
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))
plt.close('all')

acceptance ratio: 0.7615714285714286
Removed no burn in
Marginalized limits: 0.68; 0.95; 0.997

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     2.9682896E-01  9.9667276E-03  2.8597452E-01  3.0744022E-01  two    2.7860929E-01  3.1404300E-01  two    2.7056809E-01  3.2246646E-01  two     \Omega_m
hr_d        1.0165276E+02  7.9894047E-01  1.0083794E+02  1.0248655E+02  two    1.0015248E+02  1.0318671E+02  two    9.9595699E+01  1.0407138E+02  two     h r_d

L : 30, epsilon : 3e-3, std_dev : [1.,1.]
\Omega_m = 0.297\pm 0.010 h r_d = 101.65\pm 0.80
\Omega_m = 0.297^{+0.017}_{-0.018} h r_d = 101.7^{+1.5}_{-1.5}
\Omega_m = 0.297^{+0.026}_{-0.026} h r_d = 101.7^{+2.4}_{-2.1}


### MCMC

In [96]:
def proposal(x_curr):
    next_vec = np.random.normal(x_curr, [0.01,0.01])
    return pd.Series({'Omega_m': next_vec[0], 'hr_d': next_vec[1]})
from tqdm import tqdm
steps = 1
N = 1000
current_q = pd.Series({'Omega_m' : 0.2975, 'hr_d' : 101.4})+ pd.Series({'Omega_m' : np.random.normal(0,0.01), 'hr_d' : np.random.normal(0,0.1)})
samples = []; loglikes = []
accepts = 0
sampler = mh_sampler(U, proposal, steps, initial=current_q)
for i in tqdm(range(0, N, 1)):
    sample, loglike, accept = next(sampler)
    samples.append(sample)
    loglikes.append(loglike)
    accepts += accept
    q = samples[-1]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:50<00:00, 19.99it/s]


In [7]:
print(accepts)
samples = pd.DataFrame(samples)
plt.figure(figsize=(8,6))
plt.scatter(samples['Omega_m'], samples['hr_d'])
plt.savefig('s')

0.3755


In [124]:
from getdist import MCSamples, plots

samples = pd.read_csv('Practices/mcmc/MH_DESI_samples.csv')
loglikes = np.load('Practices/mcmc/MH_DESI_loglikes.npy')
accepts = np.loadtxt('Practices/mcmc/MH_DESI_acceptance_num.txt')
print('acceptance ratio:', accepts)
MCsamples_arr = np.array([samples['Omega_m'].values, samples['hr_d'].values]).T

chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'], ignore_rows=100)
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, 'Omega_m', 'hr_d', fill=True,lims=[0.265, 0.33,98.5,104.5])#lims=[0.288,0.32,100.3, 101.3]
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_MH_2d.pdf')
stats = chains.getMargeStats()
print(stats)
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))

acceptance ratio: 0.347
Marginalized limits: 0.68; 0.95; 0.99

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     2.9123363E-01  3.0063690E-03  2.8793915E-01  2.9403411E-01  two    2.8569752E-01  2.9774757E-01  two    2.8365550E-01  2.9841948E-01  two     \Omega_m
hr_d        1.0211131E+02  7.0792200E-02  1.0202056E+02  1.0221893E+02  two    1.0200653E+02  1.0224390E+02  two    1.0198686E+02  1.0226518E+02  two     h r_d

\Omega_m = 0.2912^{+0.0028}_{-0.0033} h r_d = 102.11^{+0.11}_{-0.091}
\Omega_m = 0.2912^{+0.0065}_{-0.0055} h r_d = 102.11^{+0.13}_{-0.10}
\Omega_m = 0.2912^{+0.0072}_{-0.0076} h r_d = 102.11^{+0.15}_{-0.12}


In [117]:
from getdist import MCSamples, plots

samples = pd.read_csv('Practices/mcmc/MH_DESI_samples_1.csv')
loglikes = np.load('Practices/mcmc/MH_DESI_loglikes_1.npy')
accepts = np.loadtxt('Practices/mcmc/MH_DESI_acceptance_num_1.txt')
print('acceptance ratio:', accepts)
MCsamples_arr = np.array([samples['Omega_m'].values, samples['hr_d'].values]).T

chains = MCSamples(samples=MCsamples_arr, loglikes=loglikes, names=['Omega_m', 'hr_d'], labels=[r'\Omega_m', r'h r_d'])
g = plots.get_subplot_plotter(width_inch=5)
g.plot_2d(chains, 'Omega_m', 'hr_d', filled=True,lims=[0.265, 0.33,98.5,104.5])
g.add_2d_scatter(chains, 'Omega_m', 'hr_d')
g.export('Practices/figs/DESI_MH_2d_1.pdf')
stats = chains.getMargeStats()
print(stats)
for i in range(3):
    print(chains.getInlineLatex('Omega_m', limit=i+1), chains.getInlineLatex('hr_d', limit=i+1))

acceptance ratio: 0.3679
Removed no burn in


Marginalized limits: 0.68; 0.95; 0.99

parameter   mean           sddev          lower1         upper1         limit1 lower2         upper2         limit2 lower3         upper3         limit3 
Omega_m     2.9087923E-01  4.1287083E-03  2.8652499E-01  2.9484939E-01  two    2.8317408E-01  2.9924721E-01  two    2.8117835E-01  3.0181527E-01  two     \Omega_m
hr_d        1.0214555E+02  2.3768214E-01  1.0190303E+02  1.0246064E+02  two    1.0166599E+02  1.0250973E+02  two    1.0161466E+02  1.0256723E+02  two     h r_d

\Omega_m = 0.2909\pm 0.0041 h r_d = 102.15^{+0.32}_{-0.24}
\Omega_m = 0.2909^{+0.0084}_{-0.0077} h r_d = 102.15^{+0.36}_{-0.48}
\Omega_m = 0.291^{+0.011}_{-0.0097} h r_d = 102.15^{+0.42}_{-0.53}


In [118]:
samples.shape

(30000, 2)